In [93]:
import pandas as pd
import numpy as np

In [94]:
수익성 = pd.read_csv('./datasets/코스피/코스피수익성.csv',encoding='cp949')
안정성 = pd.read_csv('./datasets/코스피/코스피 안정성.csv',encoding='cp949')
활동성 = pd.read_csv('./datasets/코스피/코스피활동성.csv',encoding='cp949')
성장성 = pd.read_csv('./datasets/코스피/코스피 성장성 계산용.csv',encoding='cp949')
현금흐름 = pd.read_csv('./datasets/코스피/코스피 현금흐름 + 누적수익성비율.csv',encoding='cp949')
연구개발 = pd.read_csv('./datasets/코스피/코스피 rs.csv',encoding='cp949')
EBIT = pd.read_csv('./datasets/코스피/에비~ 코스피.csv',encoding='cp949')
per_pbr_총부채 = pd.read_csv('./datasets/코스피/per pbr 총부채 코스피.csv',encoding='cp949')
직원수_업력 = pd.read_csv('./datasets/코스피/코스피 직원수 급여 업력 외국인 대주주.csv',encoding='cp949')
시가총액 = pd.read_csv('./datasets/코스피/코스피시가총액.csv',index_col=0)  #### 시총도 붙여야됨
단기차입금 = pd.read_csv('./datasets/코스피/코스피 단기차입금+유동성장기부채.csv',encoding='cp949')
거시변수 = pd.read_excel('./datasets/거시경제 변수/거시변수.xlsx')
상장일 = pd.read_csv('./datasets/코스피/코스피 상장일.csv',encoding='cp949')


In [95]:
df = 안정성.merge(수익성,on=['회사명','거래소코드','회계년도'],how='inner')
df = df.merge(활동성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(성장성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(현금흐름,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(연구개발,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(EBIT,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(per_pbr_총부채,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(직원수_업력,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(단기차입금,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(상장일,on=['회사명','거래소코드','회계년도'],how='left')

In [96]:
len(df.columns)

95

In [97]:
## 거래소코드 패딩
def padding(x):
    x = str(x)
    return x.rjust(6, "0")
## 외국계기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

# 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

def func_not(x):
    return not x

def change_year_2(x):
    return (x[:4])

In [98]:
## 거래소 코드 패딩
df['거래소코드'] = df['거래소코드'].map(padding)
# 회계년도 제거
df['회계년도'] = df['회계년도'].map(change_year)

In [99]:
df.isna().sum()

회사명                                               0
거래소코드                                             0
회계년도                                              0
자산(*)(요약)(IFRS)                                 170
자본(*)(요약)(IFRS)                                 170
차입금의존도(IFRS)(백만원)                               238
유보율(IFRS)                                       238
현금비율(IFRS)                                      237
부채비율(IFRS)                                      238
당좌비율(IFRS)                                      238
순운전자본비율(IFRS)                                   238
*이자비용(요약)(IFRS)(백만원)                            170
* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)                170
감가상각비(IFRS)(천원)                                 170
자산(*)(요약)(IFRS연결)                              1514
자본(*)(요약)(IFRS연결)                              1514
차입금의존도(IFRS연결)                                 1562
유보율(IFRS연결)(백만원)                               1562
현금비율(IFRS연결)(백만원)                              1561
부채비율(IFRS연결)

In [100]:
df[df.duplicated(subset=['거래소코드','회계년도'])]

,회사명,거래소코드,회계년도,자산(*)(요약)(IFRS),자본(*)(요약)(IFRS),차입금의존도(IFRS)(백만원),유보율(IFRS),현금비율(IFRS),부채비율(IFRS),당좌비율(IFRS),...,합계_기말인원(명),합계_년간총급여(천원),설립일,외국인_주식분포비율,보통주식비율,[제조] 단기차입금(요약)(IFRS),[제조] 유동성장기부채(요약)(IFRS),[제조] 단기차입금(요약)(IFRS연결),[제조] 유동성장기부채(요약)(IFRS연결),상장일
7101,평산차업집단유한공사,950010,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2002/01/03,NaN,NaN,NaN,NaN,4115.0,0.0,2007/11/26


In [101]:
## 중복되는 데이터 제거
df.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)

## 없음


In [102]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 8010 개
전체 기업 수 797


In [103]:
## 외국계 기업 제거
print("제거할 데이터 수 : ", len(df[df["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
print('제거할 외국계 기업 수',len(df[~df["거래소코드"].map(func_isnt_fcompany)].회사명.unique()))

df = df[df["거래소코드"].map(func_isnt_fcompany)]
print('남은 데이터 수',df.shape[0]) # 외국계기업 데이터 24개 제거
print('남은 기업 수',len(df['회사명'].unique()))

제거할 데이터 수 :  24
제거할 외국계 기업 수 5
남은 데이터 수 7986
남은 기업 수 792


In [104]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df["회사명"].unique()))
print("거래소코드수 : ", len(df["거래소코드"].unique()))

회사수 :  792
거래소코드수 :  795


In [105]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df["회사명"].unique()):
    if len(df[df["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")

거래소코드가 2개인 기업 수 3 개


In [106]:
list_overlap_code

['(주)신성이엔지', '삼성물산(주)', '에스케이(주)']

In [107]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df[df["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df[df["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

In [108]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(df["거래소코드"].unique()):
    if len(df[df["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

In [109]:
## 연결 결측치 개별로 채우기
for i in range(df.shape[0]):
    if pd.isnull(df.loc[i,'유보율(IFRS연결)(백만원)']):
       df.loc[i,'유보율(IFRS연결)(백만원)'] = df.loc[i,'유보율(IFRS)']
    if pd.isnull(df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'자산(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'부채(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]자본(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'당기순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융원가(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융원가(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융원가(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융수익(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융수익(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융수익(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)'] = df.loc[i,'매출액(수익)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'연구개발비(IFRS연결)(천원)']):
       df.loc[i,'연구개발비(IFRS연결)(천원)'] = df.loc[i,'연구개발비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      유형자산(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자본금회전률(IFRS연결)']):
       df.loc[i,'자본금회전률(IFRS연결)'] = df.loc[i,'자본금회전률(IFRS)']
    if pd.isnull(df.loc[i,'총자본회전률(IFRS연결)']):
       df.loc[i,'총자본회전률(IFRS연결)'] = df.loc[i,'총자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'재고자산회전률(IFRS연결)']):
       df.loc[i,'재고자산회전률(IFRS연결)'] = df.loc[i,'재고자산회전률(IFRS)']
    if pd.isnull(df.loc[i,'유형자산회전율(IFRS연결)']):
       df.loc[i,'유형자산회전율(IFRS연결)'] = df.loc[i,'유형자산회전율(IFRS)']
    if pd.isnull(df.loc[i,'자기자본회전률(IFRS연결)']):
       df.loc[i,'자기자본회전률(IFRS연결)'] = df.loc[i,'자기자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'타인자본회전률(IFRS연결)']):
       df.loc[i,'타인자본회전률(IFRS연결)'] = df.loc[i,'타인자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]경영자본순이익률(IFRS연결)']):
       df.loc[i,'[제조]경영자본순이익률(IFRS연결)'] = df.loc[i,'[제조]경영자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]금융비용부담률(IFRS연결)']):
       df.loc[i,'[제조]금융비용부담률(IFRS연결)'] = df.loc[i,'[제조]금융비용부담률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]총자본순이익률(IFRS연결)']):
       df.loc[i,'[제조]총자본순이익률(IFRS연결)'] = df.loc[i,'[제조]총자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]매출액총이익률(IFRS연결)']):
       df.loc[i,'[제조]매출액총이익률(IFRS연결)'] = df.loc[i,'[제조]매출액총이익률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]자기자본순이익률(IFRS연결)']):
       df.loc[i,'[제조]자기자본순이익률(IFRS연결)'] = df.loc[i,'[제조]자기자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]총자본정상영업이익률(IFRS연결)']):
       df.loc[i,'[제조]총자본정상영업이익률(IFRS연결)'] = df.loc[i,'[제조]총자본정상영업이익률(IFRS)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)'] = df.loc[i,'자산(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'자본(*)(요약)(IFRS연결)']):
       df.loc[i,'자본(*)(요약)(IFRS연결)'] = df.loc[i,'자본(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'현금비율(IFRS연결)(백만원)']):
       df.loc[i,'현금비율(IFRS연결)(백만원)'] = df.loc[i,'현금비율(IFRS)']
    if pd.isnull(df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'차입금의존도(IFRS연결)']):
       df.loc[i,'차입금의존도(IFRS연결)'] = df.loc[i,'차입금의존도(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'감가상각비(IFRS연결)(천원)']):
       df.loc[i,'감가상각비(IFRS연결)(천원)'] = df.loc[i,'감가상각비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'부채비율(IFRS연결)']):
       df.loc[i,'부채비율(IFRS연결)'] = df.loc[i,'부채비율(IFRS)']
    if pd.isnull(df.loc[i,'당좌비율(IFRS연결)']):
       df.loc[i,'당좌비율(IFRS연결)'] = df.loc[i,'당좌비율(IFRS)']
    if pd.isnull(df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)']):
       df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)'] = df.loc[i,'*이자비용(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'순운전자본비율(IFRS연결)']):
       df.loc[i,'순운전자본비율(IFRS연결)'] = df.loc[i,'순운전자본비율(IFRS)']
    if pd.isnull(df.loc[i, '[제조]      유동성장기부채(요약)(IFRS연결)']):
       df.loc[i, '[제조]      유동성장기부채(요약)(IFRS연결)'] = df.loc[i,'[제조]      유동성장기부채(요약)(IFRS)']
    if pd.isnull(df.loc[i,'[제조]      단기차입금(요약)(IFRS연결)']):
       df.loc[i,'[제조]      단기차입금(요약)(IFRS연결)'] = df.loc[i,'[제조]      단기차입금(요약)(IFRS)']


In [110]:
drop_col = ['유보율(IFRS)' ,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)','[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)',
'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS)(백만원)', 
'부채(*)(요약)(IFRS)(백만원)','[제조]자본(*)(요약)(IFRS)(백만원)','당기순이익(손실)(요약)(IFRS)(백만원)', 
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)','금융원가(요약)(IFRS)(백만원)','금융수익(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)','연구개발비(IFRS)(천원)',
'[제조]      유형자산(요약)(IFRS)(백만원)','자본금회전률(IFRS)',
'총자본회전률(IFRS)','재고자산회전률(IFRS)','유형자산회전율(IFRS)','자기자본회전률(IFRS)','타인자본회전률(IFRS)',
'[제조]경영자본순이익률(IFRS)','[제조]금융비용부담률(IFRS)','[제조]총자본순이익률(IFRS)','[제조]매출액총이익률(IFRS)',
'[제조]자기자본순이익률(IFRS)','[제조]총자본정상영업이익률(IFRS)','자산(*)(요약)(IFRS)','자본(*)(요약)(IFRS)',
 '현금비율(IFRS)','* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)',
 '차입금의존도(IFRS)(백만원)','감가상각비(IFRS)(천원)','부채비율(IFRS)',
'당좌비율(IFRS)','*이자비용(요약)(IFRS)(백만원)','순운전자본비율(IFRS)',
'[제조]자산(*)(요약)(IFRS)(백만원)_x','[제조]자산(*)(요약)(IFRS연결)(백만원)_x',
'[제조]자산(*)(요약)(IFRS)(백만원)_y','[제조]자산(*)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_y', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_x', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_x',
'[제조]      유동성장기부채(요약)(IFRS)','[제조]      단기차입금(요약)(IFRS)','자산(*)(요약)(IFRS연결)(백만원)','[제조]자본(*)(요약)(IFRS연결)(백만원)']

In [111]:
df.drop(drop_col,axis=1,inplace=True)

In [112]:
## 성장성 지표인 증가율 컬럼 만들기
성장성_col = ['[제조]      유형자산(요약)(IFRS연결)(백만원)','매출액(수익)(요약)(IFRS연결)(백만원)',
           '자본(*)(요약)(IFRS연결)','자산(*)(요약)(IFRS연결)',
           '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']

In [113]:
## 증가율 컬럼 전기에 음수일때 당기에 음수지만 증가할때 증가율 계산
## ex (2011년 -20, 2012년 -10) -> -10-(-20) / np.abs(-20)= 
증가율_df = (df.groupby(['회사명', '거래소코드'])[성장성_col].shift(0) - df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) *100
증가율_df.rename(columns={'[제조]      유형자산(요약)(IFRS연결)(백만원)':'유형자산증가율', '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액증가율',
       '자본(*)(요약)(IFRS연결)':'자기자본증가율', '자산(*)(요약)(IFRS연결)':'총자본증가율',
       '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'정상영업이익증가율'},inplace=True)

df.drop(['[제조]      유형자산(요약)(IFRS연결)(백만원)','[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'],axis=1,inplace=True)
df = pd.concat([df,증가율_df],axis=1)

In [114]:
## 시가총액 merge
시가총액['회계년도'] = 시가총액['회계년도'].astype(str).str[:4].astype(int)
df = df.merge(시가총액,on=['거래소코드','회계년도'],how='left')
df['시가총액(백만원)'] = df['시가총액'] / 1000000
df.drop(['시가총액'],axis=1,inplace=True)

## 거시변수 merge
거시변수.rename(columns={'회계연도':'회계년도'},inplace=True)
df = df.merge(거시변수,on=['회계년도'],how='left')

In [115]:
## 기업별 업력 설립일 이후 지속된 기간
df['업력'] = df['회계년도'] -  pd.to_datetime(df['설립일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df.drop('설립일',axis=1,inplace=True)

## 누적수익성비율
df['누적수익성비율'] = df['[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']/df['자산(*)(요약)(IFRS연결)']

## 이자부담율
df['이자부담률'] = df['EBITDA(IFRS)(백만원)'] / df['*이자비용(요약)(IFRS연결)(백만원)'] 

## 부채상환계수
df['부채상환계수'] = df['EBITDA(IFRS)(백만원)'] / (df['*이자비용(요약)(IFRS연결)(백만원)']+df['[제조]      단기차입금(요약)(IFRS연결)'] + df['[제조]      유동성장기부채(요약)(IFRS연결)'])

## 현금흐름_자산 = 영업현금흐름 / 전기총자산
df['현금흐름 대 자산'] = df.groupby(['회사명','거래소코드'])['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'].shift(0) / df.groupby(['회사명','거래소코드'])['자산(*)(요약)(IFRS연결)'].shift(1)

## 현금흐름_매출액
df['현금흐름 대 매출액'] = df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

# 토빈Q
df['토빈Q'] = (df['부채(*)(요약)(IFRS연결)(백만원)'] + df['시가총액(백만원)']) / df['자산(*)(요약)(IFRS연결)']

## TMD
df['TMD'] = df['부채(*)(요약)(IFRS연결)(백만원)'] / df['시가총액(백만원)']

## 알트만 변수
df['시총/총자산'] = df['시가총액(백만원)'] / df['자산(*)(요약)(IFRS연결)']

## RD -단위도 수정
df['연구개발비(IFRS연결)(백만원)'] = df['연구개발비(IFRS연결)(천원)']/1000
df.drop(['연구개발비(IFRS연결)(천원)'],axis=1,inplace=True)
df['RD'] = df['연구개발비(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

## 직원수변동률 , 직원인건비총액(백만원단위)
df['직원수변동률%'] = df.groupby(['회사명','거래소코드'])['합계_기말인원(명)'].pct_change()*100
df['직원인건비총액(백만원)'] = (df['합계_년간총급여(천원)']/1000) * df['합계_기말인원(명)']
df.drop(['합계_년간총급여(천원)','합계_기말인원(명)'],axis=1,inplace=True)

## 알트만 변수  EBIT = 법인세비용차감전순이익+금융원가-금융수익
df['EBIT/총자산'] = (df['법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] + df['금융원가(요약)(IFRS연결)(백만원)']-df['금융수익(요약)(IFRS연결)(백만원)'])/df['자산(*)(요약)(IFRS연결)']

## PBR , PER
df['PBR'] = df['시가총액(백만원)'] / df['당기순이익(손실)(요약)(IFRS연결)(백만원)']
df['PER'] = df['시가총액(백만원)']  / df['자산(*)(요약)(IFRS연결)']

In [116]:
len(df.columns)

65

In [117]:
rename_list = {'자산(*)(요약)(IFRS연결)':'자산', '자본(*)(요약)(IFRS연결)':'자본',
       '차입금의존도(IFRS연결)':'차입금의존도', '유보율(IFRS연결)(백만원)':'유보율', '현금비율(IFRS연결)(백만원)':'현금비율',
       '부채비율(IFRS연결)':'부채비율', '당좌비율(IFRS연결)':'당좌비율', '순운전자본비율(IFRS연결)':'순운전자본비율',
       '*이자비용(요약)(IFRS연결)(백만원)':'이자비용', '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'영업손익',
       '감가상각비(IFRS연결)(천원)':'감가상각비', 'EBITDA(IFRS)(백만원)':'EBITDA', '[제조]총자본정상영업이익률(IFRS연결)':'총자본정상영업이익률',
       '[제조]자기자본순이익률(IFRS연결)':'자기자본순이익률', '[제조]총자본순이익률(IFRS연결)':'총자본순이익률', '[제조]매출액총이익률(IFRS연결)':'매출액총이익률',
       '[제조]금융비용부담률(IFRS연결)':'금융비용부담률', '[제조]경영자본순이익률(IFRS연결)':'경영자본순이익률', '자본금회전률(IFRS연결)':'자본금회전률',
       '유형자산회전율(IFRS연결)':'유형자산회전율', '재고자산회전률(IFRS연결)':'재고자산회전률', '자기자본회전률(IFRS연결)':'자기자본회전률',
       '타인자본회전률(IFRS연결)':'타인자본회전률', '총자본회전률(IFRS연결)':'총자본회전률', 
       '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)':'이익잉여금',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)':'영업활동으로 인한 현금흐름',
       '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액', '금융원가(요약)(IFRS연결)(백만원)':'금융원가',
       '금융수익(요약)(IFRS연결)(백만원)':'금융수익', '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)':'법인세비용차감전순이익',
       '당기순이익(손실)(요약)(IFRS연결)(백만원)':'당기순이익', 
       '부채(*)(요약)(IFRS연결)(백만원)':'부채', '외국인_주식분포비율':'외국인지분율', '보통주식비율':'대주주지분율',
       '[제조]      단기차입금(요약)(IFRS연결)':'단기차입금', '[제조]      유동성장기부채(요약)(IFRS연결)':'유동성장기부채',
       '시가총액(백만원)':'시가총액','연구개발비(IFRS연결)(백만원)':'연구개발비','직원인건비총액(백만원)':'직원인건비총액',
       '통계청 한국표준산업분류 10차(대분류)':'산업군'}

df.rename(columns=rename_list,inplace=True)

In [118]:
df.isna().sum()

회사명                  0
거래소코드                0
회계년도                 0
자산                 145
자본                 145
차입금의존도             213
유보율                213
현금비율               212
부채비율               213
당좌비율               213
순운전자본비율            213
이자비용               145
영업손익               145
감가상각비              145
EBITDA             213
산업군                  0
총자본정상영업이익률         213
자기자본순이익률           213
총자본순이익률            213
매출액총이익률            213
금융비용부담률            213
경영자본순이익률           213
자본금회전률             213
유형자산회전율            213
재고자산회전률            213
자기자본회전률            213
타인자본회전률            213
총자본회전률             213
이익잉여금              145
영업활동으로 인한 현금흐름     145
매출액                145
금융원가               145
금융수익               145
법인세비용차감전순이익        145
당기순이익              145
부채                 145
외국인지분율             495
대주주지분율             437
단기차입금              145
유동성장기부채            145
상장일                  0
유형자산증가율            937
매출액증가율             937
자기자본증가율    

In [119]:
df['상장일'] = pd.to_datetime(df['상장일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)

In [120]:
# ## 회계년도 2011년 제거
df = df[df['회계년도']>2011]

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7249 entries, 1 to 7972
Data columns (total 65 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   회사명             7249 non-null   object 
 1   거래소코드           7249 non-null   object 
 2   회계년도            7249 non-null   int64  
 3   자산              7136 non-null   float64
 4   자본              7136 non-null   float64
 5   차입금의존도          7074 non-null   float64
 6   유보율             7074 non-null   float64
 7   현금비율            7075 non-null   float64
 8   부채비율            7074 non-null   float64
 9   당좌비율            7074 non-null   float64
 10  순운전자본비율         7074 non-null   float64
 11  이자비용            7136 non-null   float64
 12  영업손익            7136 non-null   float64
 13  감가상각비           7136 non-null   float64
 14  EBITDA          7074 non-null   float64
 15  산업군             7249 non-null   object 
 16  총자본정상영업이익률      7074 non-null   float64
 17  자기자본순이익률        7074 non-null   f

-----

### Y 부실 붙이기

-----

In [122]:
부실 = pd.read_csv('./datasets/y/y값최종.csv',index_col=0)

In [123]:
부실['회계년도'] = 부실['회계년도'].astype(int)
부실['거래소코드'] = 부실['거래소코드'].map(padding)
부실.drop(['번호','회사명'],axis=1,inplace=True)

In [124]:
df = df.merge(부실,on=['회계년도','거래소코드'],how='left')

In [125]:
df["공시제목"].fillna(0,inplace=True)

ind_1 = df[df["공시제목"] != 0].index

df.loc[ind_1,"공시제목"] = 1

df["공시제목"].value_counts()

for i in df.index:
    if (df.loc[i,"공시제목"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) :
        df.loc[i-1, "공시제목"] = 1

df.rename(columns={'공시제목':'target'},inplace=True)


In [126]:
def get_index_with_50_or_more_nan_columns(df):
    """
    데이터프레임에서 결측치인 컬럼의 개수가 50개 이상인 인덱스 번호를 출력하는 함수
    :param df: 데이터프레임
    :return: 결측치 컬럼 개수가 50개 이상인 인덱스 번호 리스트
    """
    # 결측치 컬럼 개수 계산
    nan_column_counts = df.isnull().sum(axis=1)
    
    # 결측치 컬럼 개수가 50개 이상인 인덱스 번호 추출
    index_with_50_or_more_nan_columns = nan_column_counts[nan_column_counts >= 50].index.to_list()
    print('제거행 수:' , len(index_with_50_or_more_nan_columns))

    df.drop(index=index_with_50_or_more_nan_columns,inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df


In [ ]:
get_index_with_50_or_more_nan_columns(df)

In [128]:
print('부실기업데이터 수:',df[df['target']==1].shape[0])

부실기업데이터 수: 56


In [129]:
print('결측치존재하는 기업 수 : ',len(df[df.isna().any(1)]['회사명'].unique()),'개')

결측치존재하는 기업 수 :  161 개


In [139]:
pd.reset_option('display.max_rows')
# pd.set_option('display.max_rows',None)
df.isna().sum()

회사명         0
거래소코드       0
회계년도        0
자산          0
자본          0
           ..
직원인건비총액     0
EBIT/총자산    0
PBR         0
PER         0
target      0
Length: 66, dtype: int64

In [140]:
## 결측치가 62개 존재하는 모든 컬럼 제거
## 첫 상장 년도와 회계년도 같은 년도 
## 외감데이터 포함되어있었음 
df = df[df['회계년도']>df['상장일']].reset_index(drop=True)

In [141]:
print('결측치존재하는 기업 수 : ',len(df[df.isna().any(1)]['회사명'].unique()),'개')

결측치존재하는 기업 수 :  0 개


In [142]:
# 한국콜마 2013년 데이터  결측치확인
# 2012년 데이터가 존재하지 않음 증가율 
# -> 한국 콜마데이터 2013년 삭제
print(df[df['매출액증가율'].isna()])
df.drop(index=6182,axis=0,inplace=True)

print('결측치존재하는 기업 수 : ',len(df[df.isna().any(1)]['회사명'].unique()),'개')

Empty DataFrame
Columns: [회사명, 거래소코드, 회계년도, 자산, 자본, 차입금의존도, 유보율, 현금비율, 부채비율, 당좌비율, 순운전자본비율, 이자비용, 영업손익, 감가상각비, EBITDA, 산업군, 총자본정상영업이익률, 자기자본순이익률, 총자본순이익률, 매출액총이익률, 금융비용부담률, 경영자본순이익률, 자본금회전률, 유형자산회전율, 재고자산회전률, 자기자본회전률, 타인자본회전률, 총자본회전률, 이익잉여금, 영업활동으로 인한 현금흐름, 매출액, 금융원가, 금융수익, 법인세비용차감전순이익, 당기순이익, 부채, 외국인지분율, 대주주지분율, 단기차입금, 유동성장기부채, 상장일, 유형자산증가율, 매출액증가율, 자기자본증가율, 총자본증가율, 정상영업이익증가율, 시가총액, 장단기금리차, cpi, 업력, 누적수익성비율, 이자부담률, 부채상환계수, 현금흐름 대 자산, 현금흐름 대 매출액, 토빈Q, TMD, 시총/총자산, 연구개발비, RD, 직원수변동률%, 직원인건비총액, EBIT/총자산, PBR, PER, target]
Index: []

[0 rows x 66 columns]
결측치존재하는 기업 수 :  0 개


In [143]:
## 직원수변동률
## 2011년 회사상장후 임원으로 만 구성
## 본격적 직원은 2012년 부터 존재 따라서 직원수변동률 0으로 처리
df[df['직원수변동률%'].isna()][['회사명','회계년도','직원수변동률%','직원인건비총액']]
df.loc[714,'직원수변동률%']=0

In [144]:
## 직원수변동률
## 한화화인케미칼 2015년 한화케미칼에 합병 

print(df[df['직원인건비총액'].isna()][['회사명','회계년도','직원수변동률%','직원인건비총액','target']])
df.drop(index=6479,axis=0,inplace=True)

Empty DataFrame
Columns: [회사명, 회계년도, 직원수변동률%, 직원인건비총액, target]
Index: []


In [145]:
## 대주주지분율 외국인지분율 결측치인 데이터
df[df['대주주지분율'].isna()][['회사명','회계년도','대주주지분율']]

## 미래산업(주) 2013,2014,2015,년 대주주지분율 0으로 대체
## (주)풍산 2013년 대주주지분율 33.35로 dart에서 확인
## 동양건설산업 2013년  1.81로 dart에서 확인

df.loc[614,'대주주지분율']=1.81
df.loc[2513,'대주주지분율']=33.35
df.loc[3983,'대주주지분율']=0
df.loc[3984,'대주주지분율']=0
df.loc[3985,'대주주지분율']=0

In [146]:
df[df['외국인지분율'].isna()][['회사명','회계년도','외국인지분율','target']]

,회사명,회계년도,외국인지분율,target


In [147]:
## 외국인 지분율 결측치는 실제 0인 값
## 그이유는 해당 년도 근처에서 관리종목 또는 횡령 또는 상폐를 앞두고있음
## 외국인 지분율이 낮아지는게 보임
df['외국인지분율'].fillna(0,inplace=True)

In [165]:
pd.set_option('display.max_columns',None)
df.describe()

,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,이자비용,영업손익,감가상각비,EBITDA,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,매출액,금융원가,금융수익,법인세비용차감전순이익,당기순이익,부채,외국인지분율,대주주지분율,단기차입금,유동성장기부채,상장일,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,시가총액,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,PER
count,6700.000000,6.700000e+03,6.700000e+03,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6700.000000,6700.000000,6.700000e+03,6.700000e+03,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6.700000e+03,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6.700000e+03,6700.000000,6700.000000,6.700000e+03,6700.000000,6700.000000,6700.000000
mean,2016.609104,3.909300e+06,1.796615e+06,23.793403,2561.514627,39.138218,167.565339,154.558003,12.554139,3.074989e+04,1.872810e+05,2.718819e+07,2.066383e+05,3.343587,-1.162473,1.617407,19.012446,1.060199,1.991749,47.044128,9.390324,275.180776,2.255294,2.431009,0.942636,1.274292e+06,2.707389e+05,2.902269e+06,8.056632e+04,6.030862e+04,1.721767e+05,1.304337e+05,2.112685e+06,10.010373,44.146325,2.335635e+05,1.668609e+05,1993.246716,38.653457,18.165173,12.037671,7.170803,38.420190,1.809702e+06,1.061248,96.959529,39.871343,0.248381,210.371218,149.080634,0.047735,0.046351,1.142650,2.114314,0.668973,4.908305e+04,0.009121,3.743425,3.449125e+09,0.028716,23.422356,0.668973
std,2.879509,1.811329e+07,1.000494e+07,17.643171,7661.741142,102.978571,1181.270702,233.426342,21.806580,1.176111e+05,1.640889e+06,1.925885e+08,1.644939e+06,6.780386,54.383377,10.433488,17.910046,3.962383,14.991812,200.440572,73.622446,10741.886883,2.286351,1.945143,0.521896,9.327772e+06,2.220047e+06,1.197052e+07,4.485274e+05,4.368279e+05,1.729693e+06,1.319643e+06,9.782949e+06,13.060500,16.773324,8.813905e+05,7.601834e+05,13.837979,1142.660163,588.949868,194.110363,40.213098,1810.302041,1.270961e+07,0.681533,3.274525,19.457154,0.431682,3791.593547,3673.226698,0.086927,0.450432,0.831659,4.247895,0.864690,6.712753e+05,0.021524,62.894654,4.817219e+10,0.101171,484.957565,0.864690
min,2012.000000,7.110000e+03,-2.666885e+06,0.000000,-2347.490000,0.000000,0.000000,3.970000,-149.650000,0.000000e+00,-5.860143e+06,0.000000e+00,-3.435112e+06,-72.880000,-2195.070000,-141.530000,-480.680000,-164.820000,-255.810000,0.100000,0.010000,0.000000,0.000000,0.010000,0.000000,-4.477062e+06,-3.480396e+06,2.234000e+03,0.000000e+00,0.000000e+00,-7.085214e+06,-5.229225e+06,5.650000e+02,0.000000,0.000000,0.000000e+00,0.000000e+00,1956.000000,-99.962978,-99.686359,-1229.612403,-96.210010,-38730.434783,5.572724e+03,0.210000,91.815000,1.000000,-5.744726,-3616.650000,-3282.390000,-1.541210,-30.937151,0.273894,0.008365,0.010280,0.000000e+00,0.000000,-99.714286,0.000000e+00,-1.202215,-11056.071859,0.010280
25%,2014.000000,2.098108e+05,1.068402e+05,8.290000,303.007500,8.447500,43.665000,64.860000,-1.542500,7.895000e+02,2.427250e+03,3.535490e+05,5.108545e+03,0.920000,-0.250000,-0.140000,9.347500,-0.070000,-0.182500,7.570000,1.710000,5.150000,1.070000,1.300000,0.610000,3.194225e+04,2.460250e+03,1.724495e+05,1.509000e+03,8.300000e+02,2.535000e+02,-4.852500e+02,7.755800e+04,1.490000,32.090000,7.299750e+03,0.000000e+00,1984.000000,-3.810221,-5.558964,-0.876984,-2.248411,-35.627242,8.984816e+04,0.510000,94.196000,27.000000,0.099934,1.049324,0.065632,0.010165,0.011517,0.770218,0.417755,0.241264,0.000000e+00,0.000000,-3.978073,1.508457e+06,0.008418,-0.405794,0.241264
50%,2017.000000,5.171600e+05,2.503075e+05,22.870000,990.260000,19.340000,90.575000

In [164]:
## inf값 존재  - 이자비용=0일때 나옴
# 이자부담률  = EBITDA / 이자비용 = 0으로 대체
# 부채상환계수  = EBITDA / (이자비용+단기차입금+유동성장기부채) = 0 으로대체
df.replace(np.inf,0,inplace=True)
df.replace(-np.inf,0,inplace=True)

In [161]:
# PBR = 시가총액 / 당기순이익 - (주)플레이그램 2013년
df[df['당기순이익']==0]
df.loc[2533,'당기순이익'] = 472540/1000000
df.loc[2533,'PBR'] = 13310.736064/(472540/1000000)

In [166]:
df.describe()

,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,이자비용,영업손익,감가상각비,EBITDA,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,매출액,금융원가,금융수익,법인세비용차감전순이익,당기순이익,부채,외국인지분율,대주주지분율,단기차입금,유동성장기부채,상장일,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,시가총액,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,PER
count,6700.000000,6.700000e+03,6.700000e+03,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6.700000e+03,6700.000000,6700.000000,6.700000e+03,6.700000e+03,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6.700000e+03,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6700.000000,6.700000e+03,6700.000000,6700.000000,6.700000e+03,6700.000000,6700.000000,6700.000000
mean,2016.609104,3.909300e+06,1.796615e+06,23.793403,2561.514627,39.138218,167.565339,154.558003,12.554139,3.074989e+04,1.872810e+05,2.718819e+07,2.066383e+05,3.343587,-1.162473,1.617407,19.012446,1.060199,1.991749,47.044128,9.390324,275.180776,2.255294,2.431009,0.942636,1.274292e+06,2.707389e+05,2.902269e+06,8.056632e+04,6.030862e+04,1.721767e+05,1.304337e+05,2.112685e+06,10.010373,44.146325,2.335635e+05,1.668609e+05,1993.246716,38.653457,18.165173,12.037671,7.170803,38.420190,1.809702e+06,1.061248,96.959529,39.871343,0.248381,210.371218,149.080634,0.047735,0.046351,1.142650,2.114314,0.668973,4.908305e+04,0.009121,3.743425,3.449125e+09,0.028716,23.422356,0.668973
std,2.879509,1.811329e+07,1.000494e+07,17.643171,7661.741142,102.978571,1181.270702,233.426342,21.806580,1.176111e+05,1.640889e+06,1.925885e+08,1.644939e+06,6.780386,54.383377,10.433488,17.910046,3.962383,14.991812,200.440572,73.622446,10741.886883,2.286351,1.945143,0.521896,9.327772e+06,2.220047e+06,1.197052e+07,4.485274e+05,4.368279e+05,1.729693e+06,1.319643e+06,9.782949e+06,13.060500,16.773324,8.813905e+05,7.601834e+05,13.837979,1142.660163,588.949868,194.110363,40.213098,1810.302041,1.270961e+07,0.681533,3.274525,19.457154,0.431682,3791.593547,3673.226698,0.086927,0.450432,0.831659,4.247895,0.864690,6.712753e+05,0.021524,62.894654,4.817219e+10,0.101171,484.957565,0.864690
min,2012.000000,7.110000e+03,-2.666885e+06,0.000000,-2347.490000,0.000000,0.000000,3.970000,-149.650000,0.000000e+00,-5.860143e+06,0.000000e+00,-3.435112e+06,-72.880000,-2195.070000,-141.530000,-480.680000,-164.820000,-255.810000,0.100000,0.010000,0.000000,0.000000,0.010000,0.000000,-4.477062e+06,-3.480396e+06,2.234000e+03,0.000000e+00,0.000000e+00,-7.085214e+06,-5.229225e+06,5.650000e+02,0.000000,0.000000,0.000000e+00,0.000000e+00,1956.000000,-99.962978,-99.686359,-1229.612403,-96.210010,-38730.434783,5.572724e+03,0.210000,91.815000,1.000000,-5.744726,-3616.650000,-3282.390000,-1.541210,-30.937151,0.273894,0.008365,0.010280,0.000000e+00,0.000000,-99.714286,0.000000e+00,-1.202215,-11056.071859,0.010280
25%,2014.000000,2.098108e+05,1.068402e+05,8.290000,303.007500,8.447500,43.665000,64.860000,-1.542500,7.895000e+02,2.427250e+03,3.535490e+05,5.108545e+03,0.920000,-0.250000,-0.140000,9.347500,-0.070000,-0.182500,7.570000,1.710000,5.150000,1.070000,1.300000,0.610000,3.194225e+04,2.460250e+03,1.724495e+05,1.509000e+03,8.300000e+02,2.535000e+02,-4.852500e+02,7.755800e+04,1.490000,32.090000,7.299750e+03,0.000000e+00,1984.000000,-3.810221,-5.558964,-0.876984,-2.248411,-35.627242,8.984816e+04,0.510000,94.196000,27.000000,0.099934,1.049324,0.065632,0.010165,0.011517,0.770218,0.417755,0.241264,0.000000e+00,0.000000,-3.978073,1.508457e+06,0.008418,-0.405794,0.241264
50%,2017.000000,5.171600e+05,2.503075e+05,22.870000,990.260000,19.340000,90.575000

In [169]:
print('최종기업수:',len(df['회사명'].unique()))

최종기업수: 757
